**MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

**Curso: Proyecto Integrador -TC5035**

Tecnológico de Monterrey

Prof Grettel Barceló Alonso

Avance 3. Baseline

**- Equipo 37** <p>Perla Fernanda Bazán Barajas<p> Ignacio Garay Ruíz<p> David Cruz Beltrán

**- Matrículas** <p>A01420700<p> A01795753<p> A01360416

# Introducción

En las etapas previas del proyecto **FlexSAIze**, se realizó un análisis exploratorio detallado de banners publicitarios y sus adaptaciones multiformato, así como un proceso de ingeniería de características que permitió enriquecer y estructurar un conjunto de datos robusto para su uso en modelos de aprendizaje automático. Estos esfuerzos permitieron identificar patrones visuales clave, jerarquías de elementos y variables relevantes para la automatización del diseño gráfico adaptativo, alineado con los objetivos de PepsiCo en eficiencia operativa y coherencia visual.<p>

En este tercer avance, el enfoque se centra en construir un modelo de referencia (baseline) que permita evaluar la viabilidad del problema y establecer expectativas realistas sobre el rendimiento que puede lograrse incluso con algoritmos simples. Este modelo de referencia permitirá no solo validar la factibilidad técnica del enfoque, sino también estimar el potencial impacto en la reducción de tiempos de producción y errores manuales dentro de los flujos creativos de PepsiCo.<p>

El modelo baseline se construirá a partir de un subconjunto representativo de banners y metadatos visuales estructurados, permitiendo evaluar la capacidad predictiva inicial del sistema sobre las jerarquías y disposiciones de elementos gráficos. Asimismo, se definirán las métricas de evaluación más adecuadas, se analizará el riesgo de subajuste o sobreajuste, y se examinará la importancia relativa de las características utilizadas por el modelo. En conjunto, este avance busca establecer una línea base cuantitativa que funcione como referencia objetiva para evaluar la mejora de los futuros modelos de inteligencia artificial aplicados al diseño adaptativo multiformato.

# Preprocesamiento de Datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
DIR = "/content/drive/MyDrive/14. Proyecto integrador/Banners/CSV"
os.chdir(DIR)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import warnings
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("banners_metadata_4.csv")
df.head()

,file,layer_name,type,priority,x,y,width,height,z_index,canvas_width,canvas_height,kv_canvas_width,kv_canvas_height,kv_x,kv_y,kv_width,kv_height,kv_z_index,is_in_kv
0,Bubly_KV.psd,BACKGROUND,BACKGROUND,-1,0,0,1706,1706,0,1706,1706,1706.0,1706.0,0.0,0.0,1706.0,1706.0,0.0,1
1,Bubly_KV.psd,SUB HEADING,TEXT,2,381,1341,910,146,1,1706,1706,1706.0,1706.0,381.0,1341.0,910.0,146.0,1.0,1
2,Bubly_KV.psd,COPY,TEXT,2,172,219,1361,162,2,1706,1706,1706.0,1706.0,172.0,219.0,1361.0,162.0,2.0,1
3,Bubly_KV.psd,SKUs,PRODUCT,2,232,523,1273,570,3,1706,1706,1706.0,1706.0,232.0,523.0,1273.0,570.0,3.0,1
4,FoodService_KV.psd,BACKGROUND,BACKGROUND,-1,0,0,1298,1080,0,1298,1080,1298.0,1080.0,0.0,0.0,1298.0,1080.0,0.0,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 906 entries, 0 to 905
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   file              906 non-null    object 
 1   layer_name        906 non-null    object 
 2   type              906 non-null    object 
 3   priority          906 non-null    int64  
 4   x                 906 non-null    int64  
 5   y                 906 non-null    int64  
 6   width             906 non-null    int64  
 7   height            906 non-null    int64  
 8   z_index           906 non-null    int64  
 9   canvas_width      906 non-null    int64  
 10  canvas_height     906 non-null    int64  
 11  kv_canvas_width   694 non-null    float64
 12  kv_canvas_height  694 non-null    float64
 13  kv_x              600 non-null    float64
 14  kv_y              600 non-null    float64
 15  kv_width          600 non-null    float64
 16  kv_height         600 non-null    float64
 1

In [ ]:
df_transformado = df.copy()

# Nuevas variables: relative_area y aspect_ratio (banner y KV)
# Banner (siempre definidas)

df_transformado["aspect_ratio"]  = df_transformado["width"] / df_transformado["height"]
df_transformado["relative_area"] = (df_transformado["width"] * df_transformado["height"]) / (
    df_transformado["canvas_width"] * df_transformado["canvas_height"]
)

# KV (solo cuando hay datos)

df_transformado["kv_aspect_ratio"] = df_transformado["kv_width"] / df_transformado["kv_height"]
df_transformado["kv_relative_area"] = (df_transformado["kv_width"] * df_transformado["kv_height"]) / (
    df_transformado["kv_canvas_width"] * df_transformado["kv_canvas_height"]
)

# Colocar NaN en KV cuando falte algo o is_in_kv == 0

kv_base_cols = ["kv_x","kv_y","kv_width","kv_height","kv_canvas_width","kv_canvas_height","kv_z_index"]
kv_derived_cols = ["kv_aspect_ratio","kv_relative_area"]
kv_all_cols = kv_base_cols + kv_derived_cols

mask_kv_invalida = (df_transformado["is_in_kv"] == 0) | df_transformado[kv_base_cols].isna().any(axis=1)
df_transformado.loc[mask_kv_invalida, kv_all_cols] = np.nan

# Eliminamos todas las instancias de BACKGROUND que no aportan información al modelo

df_transformado = df_transformado[df_transformado["layer_name"] != "Background"]

# Label encoding de 2 categóricas: layer_name, type
# Quitamos 'file' porque solo sirve como identificador, pero antes lo guardamos en una variable "grupos"

grupos = df["file"]

df_transformado.drop(columns=["file"], inplace=True)

cat_cols = ["layer_name", "type"]
label_encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    # Ajustar solo sobre valores no nulos
    df_transformado[col] = le.fit_transform(df_transformado[col].astype(str))
    label_encoders[col] = le

# Yeo-Johnson + MinMax(0,1) a numéricas "aplicables"
#    (excluimos tamaños absolutos de canvas y de KV por su naturaleza discreta/absoluta)
excluir_transform = {
    "canvas_width","canvas_height","kv_canvas_width","kv_canvas_height",
    "priority","z_index","kv_z_index"
}

# Columnas numéricas actuales
num_cols = df_transformado.select_dtypes(include=["number"]).columns.tolist()

# No transformar dummies ni las de exclusión
cols_a_transformar = [
    c for c in num_cols
    if (c not in excluir_transform) and (c not in cat_cols)
]

# Transformación columna a columna, preservando NaN
for col in cols_a_transformar:
    serie = df_transformado[col]
    mask = serie.notna()

    # Si hay al menos dos valores para ajustar
    if mask.sum() >= 2:
        # Yeo-Johnson sin estandarizar (solo "desinclina")
        pt = PowerTransformer(method="yeo-johnson", standardize=False)
        vals_pt = pt.fit_transform(serie[mask].to_numpy().reshape(-1, 1))

        # Min-Max a [0,1]
        mm = MinMaxScaler()
        vals_scaled = mm.fit_transform(vals_pt).ravel()

        # Reemplazar manteniendo NaN
        df_transformado.loc[mask, col] = vals_scaled


# Verificación rápida
print("Shape final:", df_transformado.shape)
df_transformado.info()


Shape final: (906, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 906 entries, 0 to 905
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   layer_name        906 non-null    int64  
 1   type              906 non-null    int64  
 2   priority          906 non-null    int64  
 3   x                 906 non-null    float64
 4   y                 906 non-null    float64
 5   width             906 non-null    float64
 6   height            906 non-null    float64
 7   z_index           906 non-null    int64  
 8   canvas_width      906 non-null    int64  
 9   canvas_height     906 non-null    int64  
 10  kv_canvas_width   600 non-null    float64
 11  kv_canvas_height  600 non-null    float64
 12  kv_x              600 non-null    float64
 13  kv_y              600 non-null    float64
 14  kv_width          600 non-null    float64
 15  kv_height         600 non-null    float64
 16  kv_z_index        600

In [ ]:
df_transformado.head()

,layer_name,type,priority,x,y,width,height,z_index,canvas_width,canvas_height,...,kv_x,kv_y,kv_width,kv_height,kv_z_index,is_in_kv,aspect_ratio,relative_area,kv_aspect_ratio,kv_relative_area
0,0,0,-1,0.517812,0.650798,0.779832,0.872615,0,1706,1706,...,0.490093,0.659055,0.951526,0.947282,0.0,1,0.335606,0.894187,0.435912,0.996724
1,9,7,2,0.567188,0.919896,0.698179,0.533783,1,1706,1706,...,0.673813,0.997390,0.724934,0.267499,1.0,1,0.734976,0.130355,0.832599,0.127277
2,1,7,2,0.545657,0.702679,0.750806,0.548836,2,1706,1706,...,0.597955,0.783085,0.864261,0.284667,2.0,1,0.789011,0.206615,0.874271,0.203404
3,8,6,2,0.552373,0.765253,0.742148,0.725954,3,1706,1706,...,0.621980,0.860717,0.839741,0.561137,3.0,1,0.515972,0.510997,0.633115,0.523300
4,0,0,-1,0.517812,0.650798,0.744671,0.812319,0,1298,1080,...,0.490093,0.659055,0.846816,0.765370,0.0,1,0.375318,0.894187,0.481695,0.996724


Para la construcción del modelo baseline se realizó un proceso de preprocesamiento y transformación de los datos con el objetivo de garantizar su calidad y coherencia estructural. El conjunto original contenía información sobre la disposición de elementos gráficos en banners publicitarios, incluyendo coordenadas, tamaños, jerarquías y metadatos de composición.
<p>
En primer lugar, se eliminaron capas no informativas (como Background) y se aplicó codificación categórica (Label Encoding) a las variables de tipo y nombre de capa, permitiendo su interpretación numérica por los algoritmos de aprendizaje automático. Posteriormente, se generaron nuevas variables derivadas, como aspect_ratio y relative_area tanto del banner como del key visual (KV), que permiten capturar relaciones geométricas y proporcionales relevantes para la tarea de adaptación visual.
<p>
Para asegurar una correcta comparabilidad entre características, se aplicaron transformaciones Yeo–Johnson y Min–Max Scaling (0–1) sobre las variables numéricas continuas, reduciendo sesgos de escala y distribuciones asimétricas sin afectar la estructura original de los datos. Asimismo, se manejaron los valores faltantes de los atributos del KV asignando NaN cuando el elemento no pertenecía al key visual, evitando introducir ruido en el entrenamiento.
<p>
Como resultado, se obtuvo un conjunto de datos limpio y estructurado con 906 registros y 22 variables, listo para su uso en modelos supervisados. Este proceso permitió homogeneizar magnitudes, eliminar redundancias y resaltar patrones visuales relevantes, estableciendo una base sólida para la construcción del modelo baseline y las etapas posteriores de evaluación y mejora.

# Baseline

In [ ]:
from sklearn.model_selection import train_test_split, KFold, GroupKFold, GroupShuffleSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer

In [ ]:
# Definimos "X" y "y"
y_cols = ["x", "y", "width", "height"]
X = df_transformado.drop(columns=y_cols)
y = df_transformado[y_cols].copy()

In [ ]:
# Tomamos los grupos desde el df original, alineados al índice de df_transformado
groups = df.loc[X.index, "file"]

# ---------- 1) 80% train vs 20% temp (group-aware) ----------
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
train_idx, temp_idx = next(gss1.split(X, y, groups))

X_train = X.iloc[train_idx]
y_train = y.iloc[train_idx]
groups_train = groups.iloc[train_idx]

X_temp  = X.iloc[temp_idx]
y_temp  = y.iloc[temp_idx]
groups_temp = groups.iloc[temp_idx]

# ---------- 2) Del 20% restante → 10% val y 10% test ----------
gss2 = GroupShuffleSplit(n_splits=1, test_size=0.50, random_state=42)
val_idx, test_idx = next(gss2.split(X_temp, y_temp, groups_temp))

X_val  = X_temp.iloc[val_idx]
y_val  = y_temp.iloc[val_idx]
groups_val = groups_temp.iloc[val_idx]

X_test = X_temp.iloc[test_idx]
y_test = y_temp.iloc[test_idx]
groups_test = groups_temp.iloc[test_idx]

In [ ]:
def uniq_files(g): return g.nunique()
print("Shapes -> Train:", X_train.shape, "| Val:", X_val.shape, "| Test:", X_test.shape)
print("Files únicos -> Train:", uniq_files(groups_train),
      "| Val:", uniq_files(groups_val), "| Test:", uniq_files(groups_test))

# Asegurar que no haya fuga de banners entre splits
inter_tr_val  = set(groups_train) & set(groups_val)
inter_tr_test = set(groups_train) & set(groups_test)
inter_val_test= set(groups_val)  & set(groups_test)
print("Intersecciones de 'file' (deben ser vacías):",
      len(inter_tr_val), len(inter_tr_test), len(inter_val_test))

Shapes -> Train: (720, 18) | Val: (93, 18) | Test: (93, 18)
Files únicos -> Train: 129 | Val: 16 | Test: 17
Intersecciones de 'file' (deben ser vacías): 0 0 0


In [ ]:
def report_metrics(y_true, y_pred, tag):
    rmse = mean_squared_error(y_true, y_pred)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    print(f"[{tag}] RMSE: {rmse:.4f} | MAE: {mae:.4f} | R2: {r2:.4f}")
    return rmse, mae, r2

El modelo baseline, o modelo inicial, sirve como punto de referencia para evaluar la viabilidad del problema y medir el rendimiento mínimo que puede lograrse con un enfoque simple. El propósito principal del baseline no es alcanzar la mayor precisión posible, sino entender qué tan bien se puede desempeñar un modelo básico utilizando las características disponibles y las transformaciones realizadas durante el preprocesamiento.<p>

Contar con un baseline facilita la gestión de expectativas dentro del equipo de trabajo, ya que ofrece un punto de comparación objetivo entre un modelo simple y los futuros modelos mejorados. Además, ayuda a identificar posibles problemas tempranos, como falta de información en los datos o una alta variabilidad en las respuestas.<p>

En resumen, el baseline actúa como el piso de desempeño del proyecto: si el modelo más simple logra resultados razonables, significa que el problema es abordable y vale la pena invertir en técnicas más sofisticadas. Si, por el contrario, su rendimiento es muy bajo, podría indicar que se necesitan más datos o nuevas variables antes de continuar.

# DecisionTreeRegressor


Para este primer modelo se decidió utilizar un árbol de decisión como algoritmo baseline. Este tipo de modelo es especialmente útil cuando se trabaja con datos estructurados —como en nuestro caso, donde cada registro describe características numéricas y categóricas de los elementos que conforman un banner publicitario—.<p>

Una de las principales ventajas de los árboles de decisión es que no requieren un gran volumen de datos para generar resultados interpretables. Nuestro conjunto de datos, compuesto por alrededor de 900 registros, resulta suficiente para que el modelo pueda identificar patrones básicos entre las variables (como el tamaño, la proporción o la jerarquía de los elementos) y las coordenadas o dimensiones que queremos predecir.<p>

Además, este algoritmo ofrece un equilibrio ideal entre simplicidad y capacidad de análisis. A diferencia de otros métodos más complejos, el árbol de decisión permite visualizar de forma clara cómo el modelo toma decisiones, lo que facilita entender qué variables tienen mayor peso en la predicción. Esta característica es valiosa en un contexto creativo como el de PepsiCo, donde la interpretabilidad ayuda a validar si el modelo respeta las jerarquías visuales y lineamientos de diseño de marca.<p>

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

In [ ]:
models_dtr = {}
y_pred_val_parts = []
y_pred_test_parts = []

In [ ]:
for col in y_cols:
    print(f"Entrenando modelo para '{col}' ...")

    model = DecisionTreeRegressor(
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=5,
        random_state=42
    )

    model.fit(X_train, y_train[col])
    models_dtr[col] = model

    y_pred_val = model.predict(X_val)
    y_pred_test = model.predict(X_test)

    # Mostrar métricas para cada variable por separado
    rmse_val, mae_val, r2_val = report_metrics(y_val[col], y_pred_val, "[Validación]")
    rmse_test, mae_test, r2_test = report_metrics(y_test[col], y_pred_test, "[Test]")

    # Acumular predicciones para métricas generales si quieres
    y_pred_val_parts.append(y_pred_val.reshape(-1, 1))
    y_pred_test_parts.append(y_pred_test.reshape(-1, 1))

# Combinar predicciones para todas las variables
y_pred_val_dtr  = np.hstack(y_pred_val_parts)
y_pred_test_dtr = np.hstack(y_pred_test_parts)

# Calcular métricas combinadas si quieres imprimirlas también
rmse_val_DTR, mae_val_DTR, r2_val_DTR = report_metrics(y_val,  y_pred_val_dtr,  "DTR VAL")
rmse_test_DTR, mae_test_DTR, r2_test_DTR = report_metrics(y_test, y_pred_test_dtr, "DTR TEST")

Entrenando modelo para 'x' ...
[[Validación]] RMSE: 0.0009 | MAE: 0.0169 | R2: 0.0798
[[Test]] RMSE: 0.0013 | MAE: 0.0157 | R2: 0.5659
Entrenando modelo para 'y' ...
[[Validación]] RMSE: 0.0006 | MAE: 0.0113 | R2: 0.6455
[[Test]] RMSE: 0.0005 | MAE: 0.0132 | R2: 0.7424
Entrenando modelo para 'width' ...
[[Validación]] RMSE: 0.0036 | MAE: 0.0413 | R2: 0.7774
[[Test]] RMSE: 0.0041 | MAE: 0.0418 | R2: 0.7882
Entrenando modelo para 'height' ...
[[Validación]] RMSE: 0.0017 | MAE: 0.0297 | R2: 0.9448
[[Test]] RMSE: 0.0039 | MAE: 0.0427 | R2: 0.8791
[DTR VAL] RMSE: 0.0017 | MAE: 0.0248 | R2: 0.6119
[DTR TEST] RMSE: 0.0025 | MAE: 0.0284 | R2: 0.7439


### 🔷 1. Variable x
La variable X es la más difícil de predecir con este modelo. Su desempeño en validación es muy bajo (R² ≈ 0.08), lo que indica que el modelo no logra encontrar patrones claros para esta coordenada en ese conjunto. Sin embargo, en test el desempeño mejora bastante (R² ≈ 0.57), sugiriendo que el modelo tiene cierta capacidad para generalizar, aunque la predicción sigue siendo compleja.

### 🔷 2. Variable y
La variable Y tiene un desempeño bueno, con un R² de 0.65 en validación y 0.74 en test, mostrando que el modelo identifica patrones consistentes para predecir la posición vertical.

### 🔷 3. Variable width
La variable Width muestra buenos resultados, con un R² alrededor de 0.77-0.78 tanto en validación como en test, y errores bajos, lo que indica que el modelo captura bien las reglas que definen el ancho de los elementos.

### 🔷 4. Variable height
La variable Height es la mejor predicha, con un R² muy alto (0.94 en validación y 0.88 en test), indicando que la altura está fuertemente relacionada con otras características y que el modelo puede aprender esos patrones con mucha precisión.

### Conclusiones generales del baseline con DecisionTreeRegressor

El Decision Tree Regressor logra resultados sólidos para la mayoría de las variables, especialmente para width y height, donde el modelo generaliza bien y mantiene errores bajos. Aunque la predicción de x sigue siendo un reto, el desempeño general demuestra que el problema es viable para abordarse con aprendizaje automático.
<p>
Este baseline cumple su objetivo principal: establecer un punto de partida confiable y comprensible para futuras mejoras. Los resultados sugieren que, con modelos más complejos (por ejemplo, Random Forest o Gradient Boosting) o con la incorporación de nuevas variables visuales, podría lograrse una mejora significativa en la predicción de las posiciones horizontales y en la consistencia general del layout generado por IA.

# Importancia de las características

In [ ]:
for target, model in modelos.items():
    importances = model.feature_importances_
    importances_df = pd.DataFrame({
        "feature": X_train.columns,
        "importance": importances
    }).sort_values(by="importance", ascending=False)

    print(f"\nVariables más importantes para {target.upper()}:")
    print(importances_df.head(10))


Variables más importantes para X:
             feature  importance
4       canvas_width    0.141370
8               kv_x    0.117260
7   kv_canvas_height    0.111934
15     relative_area    0.093607
0         layer_name    0.083293
1               type    0.071896
11         kv_height    0.065383
3            z_index    0.064545
2           priority    0.046129
6    kv_canvas_width    0.043203

Variables más importantes para Y:
            feature  importance
9              kv_y    0.194564
5     canvas_height    0.150799
0        layer_name    0.145321
2          priority    0.118584
15    relative_area    0.104216
11        kv_height    0.057071
1              type    0.041229
4      canvas_width    0.029947
16  kv_aspect_ratio    0.025527
14     aspect_ratio    0.022319

Variables más importantes para WIDTH:
             feature  importance
4       canvas_width    0.180880
5      canvas_height    0.164034
15     relative_area    0.127631
0         layer_name    0.122686
10         

El modelo de árbol de decisión nos permite observar de forma directa qué variables fueron más influyentes al momento de realizar las predicciones. Este análisis ayuda a entender cómo el modelo “razona” y qué aspectos del diseño del banner considera más relevantes.

## 🔹 Variable X

Las variables con mayor importancia son canvas_width, kv_x y kv_canvas_height, lo que indica que la posición horizontal de los elementos depende fuertemente del tamaño del lienzo y de su ubicación dentro del key visual (KV). También aparecen como relevantes el tipo de capa y la prioridad, lo que sugiere que ciertas categorías de elementos tienden a ubicarse de forma consistente en el eje X.

## 🔹 Variable Y

En este caso, la característica más influyente es kv_y, seguida de canvas_height y layer_name. Esto muestra que la posición vertical está muy relacionada con la estructura del KV y la jerarquía del elemento dentro del banner. También destacan priority y relative_area, que podrían reflejar cómo los elementos más importantes tienden a ubicarse en zonas específicas del diseño.

## 🔹 Variable Width

Las variables canvas_width, canvas_height y relative_area son las más relevantes, lo que tiene sentido, ya que el ancho depende directamente del tamaño del lienzo y del área relativa que ocupa el elemento. También influyen layer_name y type, lo cual sugiere que ciertos tipos de elementos mantienen proporciones típicas dentro del diseño.

## 🔹 Variable Height

Aquí destacan canvas_height, relative_area y aspect_ratio como las más influyentes. Esto indica que la altura está determinada principalmente por las proporciones generales del banner y por la escala relativa de cada componente. En menor medida, influyen variables como priority y layer_name, lo que muestra cierta coherencia entre la jerarquía del elemento y su tamaño vertical.

# Evaluación del desempeño y riesgos del modelo

Notamos que el modelo tiene un desempeño razonable para la mayoría de las variables, especialmente para la altura (height) y el ancho (width), donde los valores de R² son bastante altos. Esto indica que el modelo logra capturar patrones importantes en los datos para estas dimensiones.<p>

Sin embargo, la predicción de la posición horizontal (x) es más compleja, con un desempeño más bajo en validación. Esto puede deberse a que la posición horizontal está influenciada por factores más sutiles o variables que no fueron incluidas en el modelo, lo que limita la capacidad del árbol para aprender patrones claros.<p>

Por otro lado, los resultados en conjunto muestran que el modelo no está sobreajustando, ya que las métricas en validación y test son bastante consistentes y no hay caídas drásticas en el desempeño. Esto es positivo, pues significa que el modelo generaliza bien a datos no vistos.<p>

En resumen, este baseline con Decision Tree Regressor nos brinda una buena base para entender el problema y nos muestra que, aunque el modelo funciona bien para la mayoría de las variables, todavía hay espacio para mejorar, sobre todo en la predicción de la posición horizontal. Para seguir avanzando, podríamos probar modelos más complejos, ingeniería de características adicional o técnicas de ensamblado para capturar mejor los patrones difíciles.

# Conclusiones

En este avance hemos consolidado un baseline sólido utilizando un modelo de Decision Tree Regressor para predecir las posiciones y dimensiones de los elementos en banners publicitarios. A partir de un análisis del conjunto de datos y un preprocesamiento, logramos transformar la información cruda en características relevantes que permiten al modelo identificar patrones significativos.
<p>
Los resultados obtenidos muestran que el modelo es capaz de predecir con buena precisión variables como el ancho y la altura de los elementos, mientras que la predicción de la posición horizontal resulta más desafiante.
<p>
El análisis de importancia de variables nos permitió entender qué características aportan más a la predicción, lo cual es clave para futuras iteraciones y mejoras del modelo. Asimismo, el modelo presenta un buen equilibrio entre precisión y capacidad de generalización, evitando problemas significativos de sobreajuste.
<p>
Estos avances nos brindan una base técnica y conceptual robusta para continuar explorando métodos más avanzados, optimizar la ingeniería de características y experimentar con algoritmos más complejos que permitan mejorar la predicción, especialmente para variables más difíciles como la posición horizontal.
<p>
Con esta entrega podemos marcar las bases para automatizar y optimizar el diseño gráfico adaptativo, alineado con los objetivos de PepsiCo, y abre el camino para futuras mejoras que incrementen la eficiencia operativa y la coherencia visual en los banners publicitarios.



# Referencias

- Chen, T., & Guestrin, C. (2016). XGBoost: A scalable tree boosting system. In Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (pp. 785–794). ACM. https://doi.org/10.1145/2939672.2939785

- Friedman, J. H. (2001). Greedy function approximation: A gradient boosting machine. Annals of Statistics, 29(5), 1189–1232. https://doi.org/10.1214/aos/1013203451

- Ho, T. K. (1995). Random decision forests. In Proceedings of the 3rd International Conference on Document Analysis and Recognition (Vol. 1, pp. 278–282). IEEE. https://doi.org/10.1109/ICDAR.1995.598994

- Quinlan, J. R. (1986). Induction of decision trees. Machine Learning, 1(1), 81–106. https://doi.org/10.1007/BF00116251